# ChatGLM-6B 介绍


ChatGLM-6B 是一个开源的、支持中英双语的对话语言模型，基于 [General Language Model (GLM)](https://github.com/THUDM/GLM) 架构，具有 62 亿参数。结合模型量化技术，用户可以在消费级的显卡上进行本地部署（INT4 量化级别下最低只需 6GB 显存）。
ChatGLM-6B 使用了和 ChatGPT 相似的技术，针对中文问答和对话进行了优化。经过约 1T 标识符的中英双语训练，辅以监督微调、反馈自助、人类反馈强化学习等技术的加持，62 亿参数的 ChatGLM-6B 已经能生成相当符合人类偏好的回答。更多信息请参考我们的[博客](https://chatglm.cn/blog)。

不过，由于ChatGLM-6B的规模较小，目前已知其具有相当多的[**局限性**](#局限性)，如事实性/数学逻辑错误，可能生成有害/有偏见内容，较弱的上下文能力，自我认知混乱，以及对英文指示生成与中文指示完全矛盾的内容。请大家在使用前了解这些问题，以免产生误解。

源仓库地址 [https://github.com/THUDM/ChatGLM-6B](https://github.com/THUDM/ChatGLM-6B)

In [ ]:
#注意硬件选择必须选择高RAM，GPU选V100即可，否则会内存不足运行失败

# 复制仓库

In [3]:
!git clone https://github.com/THUDM/ChatGLM-6B
#在jupyter内切换路径一定要使用%
%cd ChatGLM-6B

fatal: destination path 'ChatGLM-6B' already exists and is not an empty directory.
/content/ChatGLM-6B


# 安装依赖

In [5]:
!ls

api.py		    improve	   ptuning	     UPDATE.md	      web_demo_vision.py
cli_demo.py	    LICENSE	   README_en.md      utils.py
cli_demo_vision.py  limitations    README.md	     web_demo2.py
examples	    MODEL_LICENSE  requirements.txt  web_demo_old.py
FAQ.md		    PROJECT.md	   resources	     web_demo.py


In [7]:
!cat requirements.txt

protobuf
transformers==4.27.1
cpm_kernels
torch>=1.10
gradio
mdtex2html
sentencepiece
accelerate

In [6]:
# Downloading gradio-4.29.0-py3-none-any.whl (12.3 MB)
!pip install -r requirements.txt
# !pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 103.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.6/416.6 kB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 109.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 88.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# 启动webui

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [1]:
from transformers import AutoModel, AutoTokenizer
# import gradio as gr

#这两句是直接从huggingface下载权重
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True).quantize(4).half().cuda()
# tokenizer = AutoTokenizer.from_pretrained("./chatglm-6b", trust_remote_code=True)
# model = AutoModel.from_pretrained("./chatglm-6b", trust_remote_code=True).quantize(4).half().cuda()
print('-'*50)
print(model)
print('-'*50)
model = model.eval()

MAX_TURNS = 20
MAX_BOXES = MAX_TURNS * 2

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/usr/local/lib/python3.11/dist-packages/huggi

config.json:   0%|          | 0.00/773 [00:00<?, ?B/s]

Explicitly passing a `revision` is encouraged when loading a configuration with custom code to ensure no malicious code has been contributed in a newer revision.


configuration_chatglm.py:   0%|          | 0.00/4.28k [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Explicitly passing a `revision` is encouraged when loading a model with custom code to ensure no malicious code has been contributed in a newer revision.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

pytorch_model-00007-of-00008.bin:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

pytorch_model-00008-of-00008.bin:   0%|          | 0.00/1.07G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, map_location="cp

--------------------------------------------------
ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (word_embeddings): Embedding(130528, 4096)
    (layers): ModuleList(
      (0-27): 28 x GLMBlock(
        (input_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attention): SelfAttention(
          (rotary_emb): RotaryEmbedding()
          (query_key_value): QuantizedLinear(in_features=4096, out_features=12288, bias=True)
          (dense): QuantizedLinear(in_features=4096, out_features=4096, bias=True)
        )
        (post_attention_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (mlp): GLU(
          (dense_h_to_4h): QuantizedLinear(in_features=4096, out_features=16384, bias=True)
          (dense_4h_to_h): QuantizedLinear(in_features=16384, out_features=4096, bias=True)
        )
      )
    )
    (final_layernorm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=409

In [ ]:



# def predict(input, history=None):
#     if history is None:
#         history = []
#     response, history = model.chat(tokenizer, input, history)
#     updates = []
#     for query, response in history:
#         updates.append(gr.update(visible=True, value="用户：" + query))
#         updates.append(gr.update(visible=True, value="ChatGLM-6B：" + response))
#     if len(updates) < MAX_BOXES: #updates做了长度限制
#         updates = updates + [gr.Textbox.update(visible=False)] * (MAX_BOXES - len(updates))
#     return [history] + updates


# with gr.Blocks() as demo:
#     state = gr.State([])
#     text_boxes = []
#     for i in range(MAX_BOXES):
#         if i % 2 == 0:
#             text_boxes.append(gr.Markdown(visible=False, label="提问："))
#         else:
#             text_boxes.append(gr.Markdown(visible=False, label="回复："))

#     with gr.Row():
#         with gr.Column(scale=4):
#             txt = gr.Textbox(show_label=False, placeholder="Enter text and press enter").style(container=False)
#         with gr.Column(scale=1):
#             button = gr.Button("Generate")
#     button.click(predict, [txt, state], [state] + text_boxes)
# demo.queue().launch(share=True, inbrowser=True)


In [2]:
def predict(input, history=None):
    if history is None:
        history = []
    response, history = model.chat(tokenizer, input, history)
    print(response)

In [4]:
predict('AI如何学习')

AI 的学习主要依赖于算法和数据。以下是一些常用的 AI 学习方法：

1. 神经网络：神经网络是一种用于学习数据的机器学习算法。可以使用公开的神经网络架构来构建自己的 AI 系统，例如使用 Transformer 架构。

2. 监督学习：监督学习是一种通过有标签的数据来学习任务的机器学习方法。可以使用公开的数据集来训练 AI 系统，并使用一些算法(如决策树)来评估 AI 的性能。

3. 无监督学习：无监督学习是一种在没有标签的数据上进行推理的机器学习方法。可以使用一些算法(如聚类、降维)来评估 AI 的性能。

4. 强化学习：强化学习是一种通过不断试错来学习策略的机器学习方法。可以使用公开的强化学习算法(如 Q-learning、 SARSA 等)来训练 AI 系统。

5. 深度学习：深度学习是一种用于学习数据的机器学习算法。可以使用公开的深度学习模型(如卷积神经网络、循环神经网络等)来构建自己的 AI 系统。

AI 的学习需要大量的数据、算法和模型来训练，并需要不断评估和改进性能。


In [ ]:
predict('类型#裙*材质#网纱*风格#性感*裙下摆#荷叶边*裙衣长#短款*裙款式#露肩')

性感网纱裙下摆设计，荷叶边和短款裙衣长，适合夏季。

材质选择网纱，既舒适又透气，可以保持身体干爽。

风格上可以选择性感和时尚，可以搭配露肩设计，突出女性优美的肩线。

这样的裙子可以吸引更多的目光，让你成为聚光灯下的焦点。


In [ ]:
#模型默认下载地址
!ls /root/.cache/huggingface/hub/models--THUDM--chatglm-6b/blobs -l

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968